In [ ]:
import pandas as pd
import plotly as pe
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# Import Data from Google Sheets 

google_sheets_url = "https://docs.google.com/spreadsheets/d/1E0lyCSxlC0ajNtzjpWo17TX5DEeEjd33E-j6c7fOBcg/export?format=csv&gid=199229195"

df_full_country_list= pd.read_csv(google_sheets_url)

df_full_country_list


In [ ]:
# --- DATA CLEANING & PIVOTING ---

# Define the filtering parameters 

START_YEAR = 2000
END_YEAR = 2023
COUNTRIES_TO_DROP = ['China', "Cote d'Ivoire", "Ghana"]

## 1. Filter by Year AND Drop Countries

# Assuming df_full_country_list is defined
df_final_working = df_full_country_list[
    (df_full_country_list['Year'] >= START_YEAR) & 
    (df_full_country_list['Year'] <= END_YEAR) &
    (~df_full_country_list['Country Name'].isin(COUNTRIES_TO_DROP))
].copy()

# 2. Pivot the filtered data once to create the single wide data source for both indices
df_wide = df_final_working.pivot_table(
    index=['Country Name', 'Year'],
    columns='Indicator Name',
    values='Value'
).reset_index()


In [ ]:
# --------------------------------------------------------------------------
#              SECTION A: ECONOMIC SUCCESS INDICATOR (ESI)
# --------------------------------------------------------------------------

Econ_indicators = [
    'GDP per capita',
    'Unemployment levels (%)',
    'Inflation (CPI, %))' 
]

# 1. Select, Rename, and Impute ESI Data
df_esi_working = df_wide[['Country Name', 'Year'] + Econ_indicators].copy()

df_esi_working.rename(columns={
    'GDP per capita': 'GDP',
    'Unemployment levels (%)': 'UNEMP',
    'Inflation (CPI, %))': 'INFLATION'
}, inplace=True)

# Imputation: FFILL and BFILL
for col in ['GDP', 'UNEMP', 'INFLATION']:
    df_esi_working[col] = df_esi_working.groupby('Country Name')[col].ffill().bfill()

# 2. Recode, Standardize, and Apply PCA
X_econ = df_esi_working[['GDP', 'UNEMP', 'INFLATION']].copy()
X_econ_recoded = X_econ.copy()
X_econ_recoded['UNEMP'] = X_econ_recoded['UNEMP'] * -1
X_econ_recoded['INFLATION'] = X_econ_recoded['INFLATION'] * -1

scaler_econ = StandardScaler()
X_econ_standardized = scaler_econ.fit_transform(X_econ_recoded)

pca_econ = PCA()
pca_econ.fit(X_econ_standardized)

# Transform 
PC_scores_esi = pca_econ.transform(X_econ_standardized)
df_esi_working['Economic Success (PCA)'] = PC_scores_esi[:, 0]


In [ ]:
# --- 1. Display the DataFrame with the new ESI Scores ---
print("--- ESI Scores per Country-Year ---")
print(df_esi_working[['Country Name', 'Year', 'Economic Success (PCA)']].head(10))

# --- 2. Display the PCA Loadings (Indicator Weights) ---
print("\n--- PCA Loadings (Weights for ESI) ---")
# The first component (index 0) holds the weights for your ESI
loadings_esi = pd.Series(pca_econ.components_[0], index=X_econ_recoded.columns)
print(loadings_esi)

# --- 3. Display the Explained Variance Ratio ---
print("\n--- Explained Variance Ratio ---")
# The first number shows the percentage of variance captured by the ESI (PC1)
print(pca_econ.explained_variance_ratio_)

In [ ]:

# --------------------------------------------------------------------------
#              SECTION B: WELL-BEING INDICATOR (WTI)
# --------------------------------------------------------------------------

WTI_indicators = [
    'Life expectancy at birth, total (years)',
    'Gini index',
    'Birth rate, crude (per 1,000 people)'
]

# 1. Select, Rename, and Impute WTI Data
df_wti_working = df_wide[['Country Name', 'Year'] + WTI_indicators].copy()

df_wti_working.rename(columns={
    'Life expectancy at birth, total (years)': 'LIFE_EXP',
    'Gini index': 'GINI',
    'Birth rate, crude (per 1,000 people)': 'BIRTH_RATE'
}, inplace=True)

# Imputation: FFILL and BFILL
for col in ['LIFE_EXP', 'GINI', 'BIRTH_RATE']:
    df_wti_working[col] = df_wti_working.groupby('Country Name')[col].ffill().bfill()
  

# 2. Recode, Standardize, and Apply PCA
X_wti = df_wti_working[['LIFE_EXP', 'GINI', 'BIRTH_RATE']].copy()

# Recode (Invert) the "bad" indicators: GINI and BIRTH_RATE
X_wti_recoded = X_wti.copy()
X_wti_recoded['GINI'] = X_wti_recoded['GINI'] * -1

scaler_wti = StandardScaler()
X_wti_standardized = scaler_wti.fit_transform(X_wti_recoded)

pca_wti = PCA()
pca_wti.fit(X_wti_standardized)

PC_scores_wti = pca_wti.transform(X_wti_standardized)
df_wti_working['Well-Being (PCA)'] = PC_scores_wti[:, 0]


In [ ]:
# --------------------------------------------------------------------------
#              SECTION C: MERGE AND FINALIZE
# --------------------------------------------------------------------------

# Merge the ESI and WTI scores into a single final DataFrame
df_merged_scores = pd.merge(
    df_esi_working[['Country Name', 'Year', 'Economic Success (PCA)']],
    df_wti_working[['Country Name', 'Year', 'Well-Being (PCA)']],
    on=['Country Name', 'Year'],
    how='left'
)

print("--- Final Merged Scores Head (WTI and ESI ready for plotting) ---")
print(df_merged_scores.head())

df_merged_scores 


In [ ]:
import plotly.express as px
import pandas as pd

# 1. CALCULATE THE AVERAGE SCORES PER COUNTRY (from yearly data)
df_final_ranking = df_merged_scores.groupby('Country Name').agg(
    {'Economic Success (PCA)': 'mean',
     'Well-Being (PCA)': 'mean'}
).reset_index()

# Rename columns for plotting simplicity
df_final_ranking.rename(columns={
    'Economic Success (PCA)': 'Avg_ESI',
    'Well-Being (PCA)': 'Avg_WTI'
}, inplace=True)


# 2. CREATE THE SCATTER PLOT
fig = px.scatter(
    df_final_ranking,
    x='Avg_ESI',           # X-axis: Economic Success Index
    y='Avg_WTI',           # Y-axis: Well-Being Translation Index
    text='Country Name',   # Label points with the country name
    title='ESI vs. WTI: Simple Comparison (2000-2023 Average)',
    labels={
        'Avg_ESI': 'Economic Success Index (ESI)',
        'Avg_WTI': 'Well-Being Index (WTI)'
    }
)

# Optional: Add Quadrant lines at the sample average (Zero)
fig.add_vline(x=0, line_dash="dash", line_color="red")
fig.add_hline(y=0, line_dash="dash", line_color="blue")

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# 1. CALCULATE THE AVERAGE ESI SCORE PER COUNTRY (from yearly data)
# This step is necessary to create the final ranking
df_ranking = df_merged_scores.groupby('Country Name')['Economic Success (PCA)'].mean().reset_index()

# Rename the column
df_ranking.rename(columns={'Economic Success (PCA)': 'Avg_ESI'}, inplace=True)

# Sort the ranking from highest ESI to lowest
df_ranking_sorted = df_ranking.sort_values(by='Avg_ESI', ascending=False)


# 2. SELECT TOP AND BOTTOM COUNTRIES FOR VISUALIZATION
df_top = df_ranking_sorted.head(10)
df_bottom = df_ranking_sorted.tail(5)
# Concatenate the top 10 and bottom 5 for contrast
df_bar_viz = pd.concat([df_top, df_bottom])

# 3. CREATE THE HORIZONTAL BAR CHART
fig = px.bar(
    df_bar_viz,
    x='Avg_ESI',
    y='Country Name',
    orientation='h', # Makes the bars horizontal (better for country names)
    color='Avg_ESI',
    color_continuous_scale=px.colors.sequential.Teal,
    title='Top & Bottom ESI Ranking (2000-2023 Average)',
    labels={'Avg_ESI': 'Average Economic Success Index (ESI) Score'}
)

# Reverse the Y-axis so the highest score is at the top
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

# Add a vertical line at ESI = 0 (the sample average)
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="red", annotation_text="Sample Average")

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# 1. CALCULATE THE AVERAGE WTI SCORE PER COUNTRY (from yearly data)
# Use 'Well-Being (PCA)' column
df_ranking_wti = df_merged_scores.groupby('Country Name')['Well-Being (PCA)'].mean().reset_index()

# Rename the column
df_ranking_wti.rename(columns={'Well-Being (PCA)': 'Avg_WTI'}, inplace=True)

# Sort the ranking from highest WTI to lowest
df_ranking_wti_sorted = df_ranking_wti.sort_values(by='Avg_WTI', ascending=False)


# 2. SELECT TOP AND BOTTOM COUNTRIES FOR VISUALIZATION
df_top_wti = df_ranking_wti_sorted.head(10)
df_bottom_wti = df_ranking_wti_sorted.tail(5)
# Concatenate the top 10 and bottom 5 for contrast
df_bar_viz_wti = pd.concat([df_top_wti, df_bottom_wti])

# 3. CREATE THE HORIZONTAL BAR CHART
fig = px.bar(
    df_bar_viz_wti,
    x='Avg_WTI',
    y='Country Name',
    orientation='h',
    color='Avg_WTI',
    color_continuous_scale=px.colors.sequential.Plasma, # Using a different color scale
    title='Top & Bottom Well-Being (WTI) Ranking (2000-2023 Average)',
    labels={'Avg_WTI': 'Average Well-Being Translation Index (WTI) Score'}
)

# Reverse the Y-axis so the highest score is at the top
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

# Add a vertical line at WTI = 0 (the sample average)
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="blue", annotation_text="Sample Average")

fig.show()